# Наследование в Python

In [1]:
class Pet:
    def __init__(self, name=None):
        self.name = name

In [2]:
# Наслідування:
class Dog(Pet):
    def __init__(self, name, breed=None):
        super().__init__(name)
        # ↑ викликає ко з батьківського класу
        self.breed = breed
        
    def say(self):
        return "{0}: waw".format(self.name)


dog = Dog('Шарік', 'Доберман')

In [3]:
dog.name

'Шарік'

In [4]:
dog.say

<bound method Dog.say of <__main__.Dog object at 0x10516f438>>

In [5]:
dog.say()

'Шарік: waw'

In [6]:
import json

class ExportJSON:
    def to_json(self):
        return json.dumps({
            'name': self.name,
            'breed': self.breed
        })
    
class ExDog(Dog, ExportJSON):
    pass

dog = ExDog('Belka', breed='Дворняжка')

In [7]:
dog.to_json()

'{"name": "Belka", "breed": "\\u0414\\u0432\\u043e\\u0440\\u043d\\u044f\\u0436\\u043a\\u0430"}'

### Любий клас є нащадком object

In [10]:
print("""int: \t\t{}
Dog: \t\t{}
Dog(Pet): \t{}
Dog <- int: \t{}
    """.format(issubclass(int, object),
              issubclass(Dog, object),
              issubclass(Dog, Pet),
              issubclass(Dog, int)))

int: 		True
Dog: 		True
Dog(Pet): 	True
Dog <- int: 	False
    


### Пошук атрибутів та методів об'єкта
#### Лінеаризація класа
```mermaid
graph TD;
    Object-->Pet;
    Object-->C(ExportJSON);
    Pet-->Dog;
    Dog-->ExDog;
    C(ExportJSON)-->ExDog;
```
![mermaid diagram](https://raw.githubusercontent.com/Searge/DiveinPython/master/img/mermaid-diagram-20190406125136.svg)

In [21]:
from pprint import pprint

# Method Resolution Order
pprint(ExDog.__mro__)

(<class '__main__.ExDog'>,
 <class '__main__.Dog'>,
 <class '__main__.Pet'>,
 <class '__main__.ExportJSON'>,
 <class 'object'>)


### Використання `super()`

In [22]:
class ExDog(Dog, ExportJSON):
    def __init_(self, name, breed=None):
        # виклик метода по MRO
        super().__init__(name, breed)
        # super(ExDog, self).__init__(name)
        
        
class WoolenDog(Dog, ExportJSON):
    def __init__(self, name, breed=None):
        # явне вказання метода конкретного класу
        super(Dog, self).__init__(name)
        self.breed = 'Шерстяна собака породи {0}'.format(breed)
        
dog = WoolenDog('Zhucka', breed='Такса')
print(dog.breed)

Шерстяна собака породи Такса


### Вирішення конфліктів імен, name mangling

In [23]:
class Dog(Pet):
    def __init__(self, name, breed=None):
        super().__init__(name)
        self.__breed = breed
        
    def say(self):
        return "{0}: waw".format(self.name)
    
    def get_breed(self):
        return self.__breed
    
class ExDog(Dog, ExportJSON):
    def get_breed(self):
        return 'Порода: {} — {}'.format(self.name,
                                       self.__breed)

In [24]:
dog = ExDog('Fox', 'Mops')
dog.__dict__

{'name': 'Fox', '_Dog__breed': 'Mops'}

In [25]:
dog.get_breed()

AttributeError: 'ExDog' object has no attribute '_ExDog__breed'

In [26]:
class ExDog(Dog, ExportJSON):
    def get_breed(self):
        return 'Порода: {} — {}'.format(self.name,
                                       self._Dog__breed)
    
dog = ExDog('Fox', 'Mops')
dog.get_breed()

'Порода: Fox — Mops'